In [1]:
import torch
import numpy as np
import copy
import matplotlib.pyplot as plt

from torch.utils.data import Dataset


from utilities import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
# batch_size = 20
batch_size = 128

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [4]:
args = dict()

args['dim_z'] = 5

In [7]:
X_train = train_data.data
X_train_left = X_train[:,:,:14]
X_train_right = X_train[:,:,14:]
Y_train = train_data.targets
X_test = test_data.data
X_test_left = X_test[:,:,:14]
X_test_right = X_test[:,:,14:]
Y_test = test_data.targets

In [8]:
X_train_left = X_train_left.numpy().reshape(len(X_train),-1)
X_train_right = X_train_right.numpy().reshape(len(X_train),-1)
X_test_left = X_test_left.numpy().reshape(len(X_test),-1)
X_test_right = X_test_right.numpy().reshape(len(X_test),-1)

In [9]:
X_train_data = list(zip(torch.Tensor(X_train_left).to(device),torch.Tensor(X_train_right).to(device)))
X_test_data = list(zip(torch.Tensor(X_test_left).to(device),torch.Tensor(X_test_right).to(device)))

In [10]:
X_train_loader = torch.utils.data.DataLoader(X_train_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True, shuffle=True)
X_test_loader = torch.utils.data.DataLoader(X_test_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True, shuffle=False)

In [11]:
class BilinearFenchelInfoNCEOne(nn.Module):
    def __init__(self,
         critic: nn.Module, 
         u_func: Optional[nn.Module] = None,
         K: Optional[int] = None,
         args: Optional[Dict] = None,
         cuda: Optional[int] = None) -> None:
        
        super(BilinearFenchelInfoNCEOne,self).__init__()
        self.critic = critic
        self.u_func = u_func
        self.K = K
    def forward(self, x, y, y0=None,K=None):
        
        '''
        x:    n x p
        y:    n x d true
        y0:   n x d fake 
        '''
        if K is None:
            K = self.K 
#         g = self.critic(x, y)
#         g0 = self.critic(x, y0)
#         u  = self.u_func(x, y)
#         output = u + torch.exp(-u+g0-g) - 1
        output  = self.PMI(x,y,y0,K)
        output = torch.clamp(output,-5,15)
        return output.mean()
    
    def MI(self, x, y, K=10):
        mi = 0
        for k in range(K):
            y0 = y[torch.randperm(y.size()[0])]
            mi += self.forward(x,y,y0)
            
        return -mi/K      
    def PMI(self, x, y, y0=None, K=None):
        '''
        x:    n x p
        y:    n x d true
        y0:   n x d fake 
        '''

        if self.u_func is not None:
            # two func mode
            u  = self.u_func(x, y)
            if K is not None:
            
                for k in range(K-1):

                    if k==0:
                        y0 = y0
                        g0 = self.critic(x, y0)
                    else:
                        y0 = y[torch.randperm(y.size()[0])]
                        g0 = torch.cat((g0,self.critic(x, y0)),1)

                g0_logsumexp = torch.logsumexp(g0,1).view(-1,1)
                output = u + torch.exp(-u+g0_logsumexp-g)/(K-1) - 1
            else:               
                
                g = self.critic(x, y)
                g0 = self.critic(x, y0)
               
                output = u + torch.exp(-u+g0-g) - 1
        else:
            # one func mode
            gu = self.critic(x,y)
            if isinstance(gu, tuple):
                hx,hy,u = gu
                similarity_matrix = hx @ hy.t()
                pos_mask = torch.eye(hx.size(0),dtype=torch.bool)
                g = similarity_matrix[pos_mask].view(hx.size(0),-1)
                g0 = similarity_matrix[~pos_mask].view(hx.size(0),-1)
                g0_logsumexp = torch.logsumexp(g0,1).view(-1,1)
                output = u + torch.exp(-u+g0_logsumexp-g)/(hx.size(0)-1) - 1

            else:      
                g, u = torch.chunk(self.critic(x,y),2,dim=1)
                if K is not None:

                    for k in range(K-1):

                        if k==0:
                            y0 = y0
                            g0,_ = torch.chunk(self.critic(x,y0),2,dim=1)
                        else:
                            y0 = y[torch.randperm(y.size()[0])]
                            g00,_ = torch.chunk(self.critic(x,y0),2,dim=1)
                            g0 = torch.cat((g0,g00),1)

                    g0_logsumexp = torch.logsumexp(g0,1).view(-1,1)
                    output = u + torch.exp(-u+g0_logsumexp-g)/(K-1) - 1
                else:    

                    g0, _ = torch.chunk(self.critic(x,y0),2,dim=1)
                    output = u + torch.exp(-u+g0-g) - 1
        return output

In [12]:
class BilinearCritic(nn.Module):
    
    '''
    encoder_x : dx -> feature_dim
    encoder_y : dy -> feature_dim
    u_func : 2*feature_dim -> 1
    '''
    
    def __init__(self,
                 encoder_x: nn.Module,
                 encoder_y: nn.Module,
                 u_func: nn.Module,
                 tau: Optional[float] = 1.):
        
        super(BilinearCritic,self).__init__()
        self.encoder_x = encoder_x
        self.encoder_y = encoder_y
        self.u_func = u_func
        self.tau = torch.nn.Parameter(torch.Tensor([tau]))
        
    
    def forward(self, x, y, tau=None):
        if tau is None:
            tau = self.tau
        tau = torch.sqrt(tau)
        hx = self.norm(self.encoder_x(x))
        hy = self.norm(self.encoder_y(y))
        u = self.u_func(hx,hy)
        
        return hx/tau, hy/tau, u  
    
    def norm(self,z):
        return torch.nn.functional.normalize(z,dim=1)
        #return z/torch.sqrt(torch.square(z).sum(1).view(-1,1))
        #return z

In [13]:
class Wrapper(nn.Module):
    def __init__(self, func):
        super(Wrapper,self).__init__()
        self.func = func
#         self.dx = dx
#         self.dy = dy
        
    def forward(self, x, y):
        xy = torch.cat([x,y],dim=1)
        
        return self.func(xy)

In [14]:
lam = 1
p = args['dim_z']
# args = {}
args['lr'] = 1e-3
args['latent_dim'] = 100
args['num_epochs'] = 50
args["input_dim"] = 2*p
#args['batch_size']=50
feature_dim = 512
encoder_x = MLP(p, output_dim=feature_dim)
encoder_y = MLP(p, output_dim=feature_dim)
u_func = Wrapper(MLP(2*feature_dim,hidden_dim=[128]))
critic = BilinearCritic(encoder_x, encoder_y, u_func)
#critic = Wrapper(MLP(args["input_dim"],2))
# u_func = Wrapper(MLP(args["input_dim"]))
mi_model = BilinearFenchelInfoNCEOne(critic)
#mitrainer = FenchelInfoNCETrainer(mi_model)

mi_model = mi_model.to(device)

In [15]:
encoder_left = MLP(14*28, args['dim_z'])
encoder_right = MLP(14*28, args['dim_z'])

encoder_left = encoder_left.to(device)
encoder_right = encoder_right.to(device)

In [16]:
optimizer = torch.optim.Adam([*encoder_left.parameters(),
            *encoder_right.parameters(),*mi_model.parameters()],lr=1e-4)

In [20]:
# number of epochs to train the model
n_epochs = 50  # suggest training between 20-50 epochs
mi_model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    train_mi = 0.0
    ###################
    # train the model #
    ###################
#     model.train()
    mi_model.train()
    for images_left, images_right in X_train_loader:
        # load the (un)labeled data
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        z_left = encoder_left(images_left)
        z_right = encoder_right(images_right)
        mi_loss = mi_model(z_left,z_right,None)
        
        # forward pass: compute predicted outputs by passing inputs to the model
#         output = model(data)
        # calculate the loss
        total_loss = mi_loss
        # backward pass: compute gradient of the loss with respect to model parameters
        total_loss.backward()
        optimizer.step()
        # perform a single optimization step (parameter update)
        
        # update running training loss
        # train_loss += loss.item()*data.size(0)
        train_mi += -mi_loss.item()*images_left.size(0)
        
    train_mi = train_mi/len(X_train_loader.dataset)
    print('Epoch: {}\t MI: {:4f}'.format(epoch+1, train_mi))

Epoch: 1	 MI: 0.682073
Epoch: 2	 MI: 0.861502
Epoch: 3	 MI: 0.958878
Epoch: 4	 MI: 1.050609
Epoch: 5	 MI: 1.146279
Epoch: 6	 MI: 1.249408
Epoch: 7	 MI: 1.371084
Epoch: 8	 MI: 1.513079
Epoch: 9	 MI: 1.683274
Epoch: 10	 MI: 1.895716
Epoch: 11	 MI: 2.182835
Epoch: 12	 MI: 2.580726
Epoch: 13	 MI: 3.120999
Epoch: 14	 MI: 3.798469
Epoch: 15	 MI: 4.308932
Epoch: 16	 MI: 4.504958
Epoch: 17	 MI: 4.571079
Epoch: 18	 MI: 4.628751
Epoch: 19	 MI: 4.671921
Epoch: 20	 MI: 4.682244
Epoch: 21	 MI: 4.693830
Epoch: 22	 MI: 4.709953
Epoch: 23	 MI: 4.734408
Epoch: 24	 MI: 4.726752
Epoch: 25	 MI: 4.741014
Epoch: 26	 MI: 4.757334
Epoch: 27	 MI: 4.771604
Epoch: 28	 MI: 4.769650
Epoch: 29	 MI: 4.777268
Epoch: 30	 MI: 4.781098
Epoch: 31	 MI: 4.783561
Epoch: 32	 MI: 4.797763
Epoch: 33	 MI: 4.793912
Epoch: 34	 MI: 4.798767
Epoch: 35	 MI: 4.804341
Epoch: 36	 MI: 4.820443
Epoch: 37	 MI: 4.811077
Epoch: 38	 MI: 4.814849
Epoch: 39	 MI: 4.825735
Epoch: 40	 MI: 4.822990
Epoch: 41	 MI: 4.822966
Epoch: 42	 MI: 4.839050
E

In [26]:
X_train_loader = torch.utils.data.DataLoader(X_train_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=False, shuffle=False)
X_test_loader = torch.utils.data.DataLoader(X_test_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=False, shuffle=False)

In [27]:
encoder_left.eval()
encoder_right.eval()

Z_train = []
Z_test = []

for images_left, images_right in X_train_loader:
    z_left = encoder_left(images_left).detach().cpu().numpy()
    z_right = encoder_right(images_right).detach().cpu().numpy()
    z = np.concatenate([z_left,z_right],1)
    Z_train.append(z)

for images_left, images_right in X_test_loader:
    z_left = encoder_left(images_left).detach().cpu().numpy()
    z_right = encoder_right(images_right).detach().cpu().numpy()
    z = np.concatenate([z_left,z_right],1)
    Z_test.append(z)

Z_train = np.concatenate(Z_train,0)
Z_test = np.concatenate(Z_test,0)

In [28]:
data_dict = dict()
data_dict['Z_train'] = Z_train
data_dict['Y_train'] = Y_train
data_dict['Z_test'] = Z_test
data_dict['Y_test'] = Y_test
data_dict['method'] = 'FLO'

In [29]:
np.save('features/flo_d_5.npy',data_dict)